In [1]:
import os
import glob
import pandas as pd

In [2]:
# Define working directory
os.chdir("C:/Users/samya/VS Code Projects/Python/Python Projects/Spotify_hit_predictor/backend")

In [3]:
ds00 = pd.read_csv('Original_data/dataset-of-00s.csv')
ds10 = pd.read_csv('Original_data/dataset-of-10s.csv')
ds60 = pd.read_csv('Original_data/dataset-of-60s.csv')
ds70 = pd.read_csv('Original_data/dataset-of-70s.csv')
ds80 = pd.read_csv('Original_data/dataset-of-80s.csv')
ds90 = pd.read_csv('Original_data/dataset-of-90s.csv')

merge = [ds00, ds10, ds60, ds70, ds80, ds90]
df = pd.concat(merge)
df.reset_index(drop=True, inplace=True)

In [4]:
df2 = df.drop(['track', 'artist', 'uri'], axis=1)

In [5]:
#Validation of correlation between variables
import numpy as np 
df3=df2.corr()

In [6]:
# Reformat data
data = df2.values
X = data[:, 0:15]  
y = data[:, 15]

print(data.shape, X.shape, y.shape)

(41106, 16) (41106, 15) (41106,)


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10)

## Data Preprocessing

In [8]:
#Scale features (X) using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler(feature_range=(0,1)).fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test) 

X_train_scaled

array([[0.78947368, 0.52283395, 0.81818182, ..., 0.8       , 0.19905999,
        0.06508876],
       [0.54048583, 0.83094117, 0.81818182, ..., 0.8       , 0.21633373,
        0.04142012],
       [0.5       , 0.66588373, 0.81818182, ..., 0.8       , 0.20480589,
        0.03550296],
       ...,
       [0.57692308, 0.36777999, 0.81818182, ..., 0.8       , 0.15844718,
        0.09467456],
       [0.41902834, 0.16070793, 0.63636364, ..., 0.8       , 0.200772  ,
        0.04733728],
       [0.29554656, 0.24873856, 0.81818182, ..., 0.6       , 0.12021911,
        0.06508876]])

One-hot encode the labels

In [9]:
#One-hot encode output labels (y)
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

y_train_categorical

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)

## Creating and defining our Deep Learning Model Architecture

In [10]:
#Create a sequential model
from tensorflow.keras.models import Sequential
model = Sequential() 

from tensorflow.keras.layers import Dense
number_inputs = 15  

#Create hidden layers
model.add(Dense(units=14,activation='relu', input_dim=number_inputs))
model.add(Dense(units=120,activation='relu'))
model.add(Dense(units=80,activation='relu'))

#Create output layer
number_classes = 2
model.add(Dense(units=number_classes, activation='softmax')) 

Number of Hidden Nodes 
https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw#:~:text=The%20number%20of%20hidden%20neurons,size%20of%20the%20input%20layer.

In [11]:
#Model Summary
model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 14)                224       
                                                                 
 dense_1 (Dense)             (None, 120)               1800      
                                                                 
 dense_2 (Dense)             (None, 80)                9680      
                                                                 
 dense_3 (Dense)             (None, 2)                 162       
                                                                 
Total params: 11,866
Trainable params: 11,866
Non-trainable params: 0
_________________________________________________________________


In [12]:
#Compile the Model
import tensorflow as tf

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

## Training the Model

In [13]:
#Training the Model
history = model.fit(X_train_scaled, y_train_categorical, epochs=500, batch_size=2000, shuffle=True, verbose=2)

Epoch 1/500
16/16 - 0s - loss: 0.6640 - accuracy: 0.6230 - 376ms/epoch - 23ms/step
Epoch 2/500
16/16 - 0s - loss: 0.6031 - accuracy: 0.6944 - 53ms/epoch - 3ms/step
Epoch 3/500
16/16 - 0s - loss: 0.5592 - accuracy: 0.7081 - 50ms/epoch - 3ms/step
Epoch 4/500
16/16 - 0s - loss: 0.5422 - accuracy: 0.7203 - 50ms/epoch - 3ms/step
Epoch 5/500
16/16 - 0s - loss: 0.5355 - accuracy: 0.7257 - 50ms/epoch - 3ms/step
Epoch 6/500
16/16 - 0s - loss: 0.5312 - accuracy: 0.7305 - 50ms/epoch - 3ms/step
Epoch 7/500
16/16 - 0s - loss: 0.5266 - accuracy: 0.7336 - 50ms/epoch - 3ms/step
Epoch 8/500
16/16 - 0s - loss: 0.5218 - accuracy: 0.7371 - 50ms/epoch - 3ms/step
Epoch 9/500
16/16 - 0s - loss: 0.5181 - accuracy: 0.7420 - 81ms/epoch - 5ms/step
Epoch 10/500
16/16 - 0s - loss: 0.5150 - accuracy: 0.7453 - 76ms/epoch - 5ms/step
Epoch 11/500
16/16 - 0s - loss: 0.5119 - accuracy: 0.7450 - 41ms/epoch - 3ms/step
Epoch 12/500
16/16 - 0s - loss: 0.5082 - accuracy: 0.7470 - 49ms/epoch - 3ms/step
Epoch 13/500
16/16 - 0s

Epoch 101/500
16/16 - 0s - loss: 0.4839 - accuracy: 0.7649 - 42ms/epoch - 3ms/step
Epoch 102/500
16/16 - 0s - loss: 0.4833 - accuracy: 0.7652 - 35ms/epoch - 2ms/step
Epoch 103/500
16/16 - 0s - loss: 0.4831 - accuracy: 0.7662 - 41ms/epoch - 3ms/step
Epoch 104/500
16/16 - 0s - loss: 0.4847 - accuracy: 0.7633 - 46ms/epoch - 3ms/step
Epoch 105/500
16/16 - 0s - loss: 0.4834 - accuracy: 0.7650 - 37ms/epoch - 2ms/step
Epoch 106/500
16/16 - 0s - loss: 0.4824 - accuracy: 0.7665 - 41ms/epoch - 3ms/step
Epoch 107/500
16/16 - 0s - loss: 0.4830 - accuracy: 0.7663 - 33ms/epoch - 2ms/step
Epoch 108/500
16/16 - 0s - loss: 0.4842 - accuracy: 0.7631 - 42ms/epoch - 3ms/step
Epoch 109/500
16/16 - 0s - loss: 0.4838 - accuracy: 0.7642 - 41ms/epoch - 3ms/step
Epoch 110/500
16/16 - 0s - loss: 0.4829 - accuracy: 0.7661 - 42ms/epoch - 3ms/step
Epoch 111/500
16/16 - 0s - loss: 0.4821 - accuracy: 0.7668 - 33ms/epoch - 2ms/step
Epoch 112/500
16/16 - 0s - loss: 0.4822 - accuracy: 0.7660 - 41ms/epoch - 3ms/step
Epoc

Epoch 200/500
16/16 - 0s - loss: 0.4741 - accuracy: 0.7696 - 40ms/epoch - 3ms/step
Epoch 201/500
16/16 - 0s - loss: 0.4736 - accuracy: 0.7696 - 34ms/epoch - 2ms/step
Epoch 202/500
16/16 - 0s - loss: 0.4728 - accuracy: 0.7706 - 42ms/epoch - 3ms/step
Epoch 203/500
16/16 - 0s - loss: 0.4728 - accuracy: 0.7703 - 41ms/epoch - 3ms/step
Epoch 204/500
16/16 - 0s - loss: 0.4735 - accuracy: 0.7701 - 42ms/epoch - 3ms/step
Epoch 205/500
16/16 - 0s - loss: 0.4740 - accuracy: 0.7699 - 33ms/epoch - 2ms/step
Epoch 206/500
16/16 - 0s - loss: 0.4743 - accuracy: 0.7700 - 41ms/epoch - 3ms/step
Epoch 207/500
16/16 - 0s - loss: 0.4767 - accuracy: 0.7678 - 51ms/epoch - 3ms/step
Epoch 208/500
16/16 - 0s - loss: 0.4737 - accuracy: 0.7696 - 41ms/epoch - 3ms/step
Epoch 209/500
16/16 - 0s - loss: 0.4738 - accuracy: 0.7701 - 42ms/epoch - 3ms/step
Epoch 210/500
16/16 - 0s - loss: 0.4726 - accuracy: 0.7707 - 50ms/epoch - 3ms/step
Epoch 211/500
16/16 - 0s - loss: 0.4724 - accuracy: 0.7710 - 42ms/epoch - 3ms/step
Epoc

Epoch 299/500
16/16 - 0s - loss: 0.4663 - accuracy: 0.7741 - 52ms/epoch - 3ms/step
Epoch 300/500
16/16 - 0s - loss: 0.4673 - accuracy: 0.7731 - 51ms/epoch - 3ms/step
Epoch 301/500
16/16 - 0s - loss: 0.4664 - accuracy: 0.7742 - 53ms/epoch - 3ms/step
Epoch 302/500
16/16 - 0s - loss: 0.4657 - accuracy: 0.7739 - 45ms/epoch - 3ms/step
Epoch 303/500
16/16 - 0s - loss: 0.4652 - accuracy: 0.7741 - 48ms/epoch - 3ms/step
Epoch 304/500
16/16 - 0s - loss: 0.4646 - accuracy: 0.7748 - 43ms/epoch - 3ms/step
Epoch 305/500
16/16 - 0s - loss: 0.4656 - accuracy: 0.7753 - 46ms/epoch - 3ms/step
Epoch 306/500
16/16 - 0s - loss: 0.4661 - accuracy: 0.7757 - 42ms/epoch - 3ms/step
Epoch 307/500
16/16 - 0s - loss: 0.4666 - accuracy: 0.7738 - 44ms/epoch - 3ms/step
Epoch 308/500
16/16 - 0s - loss: 0.4646 - accuracy: 0.7751 - 112ms/epoch - 7ms/step
Epoch 309/500
16/16 - 0s - loss: 0.4648 - accuracy: 0.7759 - 126ms/epoch - 8ms/step
Epoch 310/500
16/16 - 0s - loss: 0.4661 - accuracy: 0.7759 - 119ms/epoch - 7ms/step
E

Epoch 398/500
16/16 - 0s - loss: 0.4593 - accuracy: 0.7784 - 46ms/epoch - 3ms/step
Epoch 399/500
16/16 - 0s - loss: 0.4592 - accuracy: 0.7788 - 58ms/epoch - 4ms/step
Epoch 400/500
16/16 - 0s - loss: 0.4593 - accuracy: 0.7777 - 59ms/epoch - 4ms/step
Epoch 401/500
16/16 - 0s - loss: 0.4595 - accuracy: 0.7781 - 53ms/epoch - 3ms/step
Epoch 402/500
16/16 - 0s - loss: 0.4589 - accuracy: 0.7796 - 54ms/epoch - 3ms/step
Epoch 403/500
16/16 - 0s - loss: 0.4592 - accuracy: 0.7786 - 47ms/epoch - 3ms/step
Epoch 404/500
16/16 - 0s - loss: 0.4593 - accuracy: 0.7797 - 61ms/epoch - 4ms/step
Epoch 405/500
16/16 - 0s - loss: 0.4598 - accuracy: 0.7787 - 59ms/epoch - 4ms/step
Epoch 406/500
16/16 - 0s - loss: 0.4592 - accuracy: 0.7790 - 50ms/epoch - 3ms/step
Epoch 407/500
16/16 - 0s - loss: 0.4586 - accuracy: 0.7798 - 50ms/epoch - 3ms/step
Epoch 408/500
16/16 - 0s - loss: 0.4591 - accuracy: 0.7781 - 50ms/epoch - 3ms/step
Epoch 409/500
16/16 - 0s - loss: 0.4610 - accuracy: 0.7790 - 34ms/epoch - 2ms/step
Epoc

Epoch 497/500
16/16 - 0s - loss: 0.4553 - accuracy: 0.7816 - 42ms/epoch - 3ms/step
Epoch 498/500
16/16 - 0s - loss: 0.4541 - accuracy: 0.7813 - 43ms/epoch - 3ms/step
Epoch 499/500
16/16 - 0s - loss: 0.4527 - accuracy: 0.7821 - 47ms/epoch - 3ms/step
Epoch 500/500
16/16 - 0s - loss: 0.4535 - accuracy: 0.7820 - 51ms/epoch - 3ms/step


## Validation of the Model

In [14]:
#Evaluate the Model using the testing data
#Compare Model performace between training and testing data
model_loss_train, model_accuracy_train = model.evaluate(X_train_scaled, y_train_categorical, verbose=2)
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test_categorical, verbose=2)

print(f"TRAINING DATA --> Loss: {model_loss_train}, Accuracy: {model_accuracy_train}")    
print(f"TESTING DATA --> Loss: {model_loss}, Accuracy: {model_accuracy}")

964/964 - 1s - loss: 0.4529 - accuracy: 0.7819 - 820ms/epoch - 850us/step
322/322 - 0s - loss: 0.4845 - accuracy: 0.7717 - 213ms/epoch - 663us/step
TRAINING DATA --> Loss: 0.4529109001159668, Accuracy: 0.7818612456321716
TESTING DATA --> Loss: 0.48452523350715637, Accuracy: 0.7717232704162598


## Saving the Trained Model

In [15]:
# Save the model
model.save("../Models/h5/model.h5")